In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0541, -0.2074, -0.1825, -0.0221,  0.1330,  0.1162, -0.2108,  0.2289,
         -0.1769,  0.3393],
        [-0.0068, -0.2331, -0.0070,  0.0412,  0.0749,  0.1677, -0.1459,  0.1661,
         -0.0473,  0.2284]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
    

In [3]:
net = MLP()
net(X)

tensor([[-0.1227, -0.0442, -0.1834, -0.0753,  0.2375,  0.0481, -0.3060,  0.1084,
          0.0800, -0.1283],
        [-0.0347,  0.1637, -0.1345, -0.0886,  0.2337,  0.0490, -0.2961,  0.1638,
         -0.0068, -0.0389]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args: #args有序词典
            self._modules[block] = block
        
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)  #一层层的返回值作为输入
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.3512, -0.0965,  0.0021, -0.0340, -0.2643, -0.1058, -0.3059, -0.2183,
          0.0271,  0.4202],
        [ 0.0791,  0.0605,  0.0912, -0.0564, -0.0160, -0.0323, -0.3740, -0.2144,
          0.0758,  0.3443]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(-0.1414, grad_fn=<SumBackward0>)

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1672, grad_fn=<SumBackward0>)

In [10]:
print(net)

FixedHiddenMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


In [11]:
class TwoNet(nn.Module):
    def __init__(self):
        super().__init__() #继承父类的构造函数
        self.net1 = nn.Sequential(nn.Linear(20, 30), nn.ReLU())
        self.net2 = nn.Sequential(nn.Linear(20,40), nn.ReLU())
    
    def forward(self, X):
        X1 = self.net1(X)
        X2 = self.net2(X)
        
        Output = torch.cat((X1, X2), 1)
        
        return Output
 
net = TwoNet()
        
X = torch.rand(2, 20)   
print(net(X))   
print(net(X).shape)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7474e-01, 0.0000e+00, 6.9568e-01,
         0.0000e+00, 2.4193e-01, 1.4506e-01, 0.0000e+00, 1.3523e-01, 8.2246e-01,
         2.4255e-01, 1.6334e-01, 3.6582e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         4.6088e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         2.8329e-01, 7.1025e-02, 2.4544e-01, 2.9185e-01, 2.6132e-01, 0.0000e+00,
         0.0000e+00, 2.8719e-01, 8.0270e-04, 1.1067e-01, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.7554e-01,
         0.0000e+00, 2.0157e-01, 2.5472e-04, 3.6797e-01, 5.0068e-01, 0.0000e+00,
         2.7889e-01, 4.1679e-01, 2.5969e-01, 0.0000e+00, 2.7963e-01, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 7.7285e-02,
         0.0000e+00, 8.5210e-02, 2.1057e-01, 0.0000e+00, 0.0000e+00, 1.1644e-01,
         0.0000e+00, 2.6329e-02, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.9185

In [12]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1058],
        [-0.0969]], grad_fn=<AddmmBackward0>)

In [13]:
print(net[2].state_dict()) #由于权重和偏差是可变的，故称为state, _dict理解为_modules

OrderedDict([('weight', tensor([[ 0.1341, -0.3535, -0.0370, -0.2316,  0.1430, -0.0197, -0.2441, -0.1700]])), ('bias', tensor([-0.1137]))])


In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data) #.data为值，.grad访问梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1137], requires_grad=True)
tensor([-0.1137])


In [15]:
net[2].weight.grad == None

True

In [16]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()]) 
print(*[(name, param.shape) for name, param in net.named_parameters()]) #Relu没有参数，故拿不出来

print(type(net[0].named_parameters()))

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))
<class 'generator'>


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.1137])

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1()) #这样每个块都有字符串名字，比如block1
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1906],
        [0.1907]], grad_fn=<AddmmBackward0>)

In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [20]:
#修改默认的初始化参数
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) #函数后面接下划线表示 替换函数（原地操作，不加的话相当于又复制一份），直接将m.weight值替换掉，而没有返回值
        nn.init.zeros_(m.bias)
net.apply(init_normal) #对net里面所有module循环遍历，如果满足条件就依条件初始化
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0094,  0.0005, -0.0045,  0.0022]), tensor(0.))

In [21]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) #初始化成一个值
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [22]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.3671, -0.4225, -0.0270,  0.3879])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [23]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.7030, -6.1627,  6.0424,  0.0000],
        [ 0.0000, -6.3001,  5.1143, -0.0000]], grad_fn=<SliceBackward0>)

In [24]:
#暴力初始化，直接替换
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -5.1627,  7.0424,  1.0000])

In [25]:
#不同层之间share参数
#参数绑定
# 我们需要给共享层一个名称，以便可以引用它的参数
#需要提前建立好需要共享层
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [26]:
# 自定义层
# 一样继承 nn.module
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [27]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [28]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [29]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(1.8626e-09, grad_fn=<MeanBackward0>)

In [30]:
#需要parameter类型才可以梯度优化
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) #转化为parameter
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data #前向传播需要加上 .data
        return F.relu(linear)

In [31]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 1.9731, -0.6047,  1.5496],
        [-1.5280, -0.4771, -0.9518],
        [ 1.5829,  1.1415,  1.9520],
        [ 0.1347,  0.7170,  0.8840],
        [ 0.6781, -1.8506,  0.4079]], requires_grad=True)

In [32]:
linear(torch.rand(2, 5))

tensor([[0.4608, 0.0000, 0.7068],
        [3.2871, 0.0000, 3.1559]])

In [33]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.0000],
        [0.6039]])

In [34]:
#读写文件
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [35]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [36]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [37]:
#保存dit
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [38]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [39]:
torch.save(net.state_dict(), 'mlp.params') #所有参数存储为字典

In [40]:
#load模型参数
clone = MLP()
clone.load_state_dict(torch.load('mlp.params')) #overwrite掉原来的参数，将文件中的参数导入
clone.eval() #看模型

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [41]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [42]:
!nvidia-smi

'nvidia-smi' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
